In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies)
reload(pi.strategies.buy_x_week_low)
reload(pi.strategies.buy_close)
reload(pi.summarize.trades)
reload(pi.indicators)

In [ ]:
symbols = ['A','AA']

In [ ]:
#df = gas.get_symbols_df(symbols)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
saved_df = df.clone()

In [ ]:
fluctuation = pi.indicators.fluctuation_percentage(df)
fluct_sma3 = pi.indicators.simple_moving_average(fluctuation.df, 7, fluctuation.column)
#grouped_fluctation = pi.indicators.group_by_amount_display(fluct_sma3.df, fluct_sma3.column, [1,2,3,5,7,10,20,30,40,50,60,70,80,90])
grouped_fluctation = pi.indicators.group_by_amount(fluct_sma3.df, fluct_sma3.column, [1,2,3,5,7,10,20])
df = grouped_fluctation.df

In [ ]:
relative_volume = pi.indicators.relative_volume(df, 5)
grouped_relative_volume = pi.indicators.group_by_amount(relative_volume.df, relative_volume.column, [x * 25 for x in range(1,7)])
df = grouped_relative_volume.df

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

In [ ]:
#filter out average price too low
df = df.filter(pl.col(pi.indicators.CLOSE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 3)

# filter out too many 0 volume bars. This gets rid of a lot of garbage
df = df.filter((pl.col(pi.indicators.VOLUME_COLUMN) == 0).sum().over(pi.indicators.SYMBOL_COLUMN) < 3)

# minimum amount of volume
df = df.filter(pl.col(pi.indicators.VOLUME_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 100000)


In [ ]:
# #buy_x_week_low
# lookback_bars = 7
# offset_bars = 3
# min_price = 3
# offset_percentage = 1
# extra_columns = [grouped_relative_volume.column, grouped_fluctation.column]

# roll_min = pi.indicators.rolling_min_with_offset(df, column=pi.indicators.LOW_COLUMN, bars=lookback_bars, offset=offset_bars)
# df = roll_min.df

# #min_price filter
# df = df.with_columns(pl.when(pl.col(roll_min.column) >= min_price).then(pl.col(roll_min.column)))

# #filter when bar in offset range was below minimum
# df = df.with_columns(pl.when(pl.col(roll_min.column) < pl.col(pi.indicators.LOW_COLUMN).rolling_min(offset_bars).shift().over(pi.indicators.SYMBOL_COLUMN)).then(
#     pl.col(roll_min.column)))

# #take the offset_percentage into account
# factor = (100 + offset_percentage) / 100
# df = df.with_columns(pl.col(roll_min.column) * factor)


# #this is where we differ. Just calculating when the contitions are true for passing to buy_close
# enter_column = "enter"
# target = pi.indicators.targeted_value(df, roll_min.column)
# df = target.df.with_columns((
#         pl.col(target.column).is_not_null()
#         &
#         (
#             ((pl.col(grouped_fluctation.column) >= 10) & (pl.col(grouped_relative_volume.column) == 25))
#             |
#             ((pl.col(grouped_fluctation.column) == 20.01) & (pl.col(grouped_relative_volume.column) == 50))
#         )
#     ).alias(enter_column))
# df


In [ ]:
# res = pi.strategies.buy_close.strategy(df, enter_column, 2)

In [ ]:
# ts = pi.summarize.trades.Trades.from_strategy_result(res, extra_columns)
# ts.summarize_strategy(extra_columns)

In [ ]:
# ts.df.filter(pl.col("Start") > datetime(2023,1,1))

In [ ]:
lookback_bars = 14
offset_bars = 3
modresult_05 = pi.strategies.buy_x_week_low.strategy(df, lookback_bars, offset_bars, 0.5, -5, 2, 3)
modresult_1 = pi.strategies.buy_x_week_low.strategy(df, lookback_bars, offset_bars, 1, -5, 2, 3)

In [ ]:
extra_columns = [grouped_fluctation.column, grouped_relative_volume.column]
trades_05 = pi.summarize.trades.Trades.from_strategy_result(modresult_05, extra_columns)
trades_1 = pi.summarize.trades.Trades.from_strategy_result(modresult_1, extra_columns)

In [ ]:
ss_05 = trades_05.summarize_strategy([grouped_relative_volume.column, grouped_fluctation.column]).sort("Average_Gain/Loss%")
pl.Config.set_tbl_rows(100)
ss_05

In [ ]:
ss_1 = trades_1.summarize_strategy([grouped_relative_volume.column, grouped_fluctation.column]).sort("Average_Gain/Loss%")
ss_1

In [ ]:
ss_1.join(ss_05, on=extra_columns).select(extra_columns,"Trades","Trades_right","Win_Ratio","Win_Ratio_right","Average_Gain/Loss%","Average_Gain/Loss%_right")

In [ ]:
filtered_1 = trades_1.df.filter((
    (pl.col(grouped_fluctation.column) >= 10) & (pl.col(grouped_relative_volume.column) == 25))
    |
    (pl.col(grouped_fluctation.column) == 20.01) & (pl.col(grouped_relative_volume.column) == 50))


In [ ]:
filtered_1.filter(pl.col("Start") > datetime(2023,1,1))

In [ ]:
modresult_1.df.filter(pl.col(modresult_1.ids_column) == 25790)

In [ ]:
filtered_1.filter(pl.col("Gain/Loss%") > 50).sort(pl.col("Gain/Loss%"))

In [ ]:
trades_05.df.filter(pl.col("Highest%") > 557.16)

In [ ]:
tf = trades_05.df.groupby(pi.indicators.SYMBOL_COLUMN).agg(pl.col(pi.indicators.SYMBOL_COLUMN).count().alias("trades")).sort("trades", descending=True)
tf

In [ ]:
tester = modresult_05.df.with_row_count("row").filter(pl.col("row").is_between(820,831)).select("Date","Open","High","Low","Close","Volume","Symbol")